In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from Map import Map
from Tabu import Tabu
from TwoStage import TwoStage
from L_shaped import MasterProblem as mp
import matplotlib.pyplot as plt
import matplotlib.pylab as pl 
import matplotlib.cm as cm
import time
import networkx as nx
import matplotlib as mpl
from gurobipy import *
import csv

**Modelling Parameters**

In [3]:
rides = 5
bus = 1
scenarios = 25
MIPGap = 0.001
TimeLimit = 18000
probability = [0.7, 0.1, 0.15, 0.05]

**Defining the Map**

In [4]:
mappy = Map(rides)

**Plotting Functions**

In [5]:
def plot_trend(ub,lb,base):
    x = list(range(len(ub)))
    figure = plt.figure()
    plt.plot(x,ub,label='upper-bound')
    plt.plot(x,[base]*len(ub),label='two-stage')
    plt.plot(x,lb,label='lower-bound')
    plt.annotate(int(ub[-1]),[x[-1],ub[-1]])
    plt.legend()
    plt.savefig('./figures/trend.png')
    plt.show()

colors = ['green','blue','yellow','red','pink'] 
def displaygraph(n, e, modname):
    for k in e.keys():
        for i, j in e[k]:
            plt.plot((n[i][0],n[j][0]),(n[i][1],n[j][1]), color=colors[k] ,marker='o', linewidth=2, linestyle='dashed')
    for i in n:
        plt.annotate(i,n[i],textcoords="offset points",xytext=(0,10),ha='center')
    plt.savefig('./figures/'+modname+'.png')
    plt.show()


#### Model formulation

In [6]:
def Mod1(aux=False):
    t1 = time.time()
    drpstw = mp(mappy,bus=bus,scenarios=scenarios, probability=probability)
    drpstw.initialize()
    drpstw.setMIP()
    drpstw.variables.th.obj=0
    if aux:
        drpstw.setLowerBound()
        for s in range(drpstw.scenarios):
            drpstw.submodel[s].model.params.MIPGap = MIPGap
#             drpstw.submodel[s].model.params.OutputFlag = 1
    else:
        drpstw.model.params.TimeLimit = TimeLimit
        drpstw.model.optimize()
    t1 = time.time() - t1
    return drpstw, t1

In [7]:
def Mod2(model=None):
    t2 = time.time()
    twosp = TwoStage(mappy, bus=bus, scenarios=scenarios, probability=probability)
    twosp.model.params.TimeLimit = TimeLimit
    if model is not None:
        for i,j,k in twosp.MP.variables.x.keys():
            twosp.MP.variables.x[i,j,k].start = model.variables.x[i,j,k].X
    twosp.optimize()
    t2 = time.time() - t2
    return twosp, t2

In [8]:
def Mod3():
    t3 = time.time()
    lshaped = mp(mappy, bus=bus, scenarios=scenarios, probability=probability)
    lshaped.initialize()
    lshaped.model.params.TimeLimit = TimeLimit
    lshaped.optimize()
    t3 = time.time() - t3
    return lshaped, t3

In [9]:
def Mod4(lshaped):
    t4 = time.time()
    tabu = Tabu(lshaped,tabu_iterations=200, tabu_status=10, subset=10, tsp=False)
    tabu.tsp = True
    tabu.tabuheuristic()
    t4 = time.time() - t4
    return tabu, t4

In [10]:
def Mod5(lshaped):
    t5 = time.time()
    tabutsp = Tabu(lshaped,tabu_itcerations=100, subset=10, tsp=True)
    tabutsp.tabuheuristic()
    t5 = time.time() - t5
    return tabutsp, t5

### Test Block

In [ ]:
TimeLimit = 18000
tests = [(3,2),(4,2),(5,2),(6,2),(7,2),(9,3),(10,3),(11,3),(13,4),(15,4),(18,5),(20,5),(22,6),(24,6),(25,7),(27,8)]
scenarioslist = [10,20,30]

with open('./Reports/Report.csv', 'w', newline='') as csv_file:
    write = csv.writer(csv_file, delimiter=',')
    write.writerow(('Rides','Bus','Scenarios','TSPObj','TSPGap','TSPTime','LshapeObj','LshapeSave','LshapeGap','LshapeTime','TabuObj','TabuSave','TabuTime'))
    for s in scenarioslist:
        scenarios = s
        for test in tests:
            rides, bus = test
            mappy = Map(rides)
            #Two-Stage
            try:    
                twostage, t1 = Mod2()
                if twostage.model.status in [2,9]:
                    twostageobj = twostage.model.ObjVal
                    twostagegap = twostage.model.MIPGap
                    twostagetime = t1
                else:
                    twostageobj = 'NA'
                    twostagegap = 'NA'
                    twostagetime = t1
            except GurobiError:
                twostageobj = 'Failed'
                twostagegap = 'Failed'
                twostagetime = 'Failed'
            #L-shaped
            try:    
                lshaped, t2 = Mod3()
                if lshaped.model.status in [2,9]:
                    lshapedobj = lshaped.model.ObjVal
                    lshapedsave = lshaped.variables.th.X
                    lshapedgap = lshaped.model.MIPGap
                    lshapedtime = t2
                else:
                    lshapedobj = 'NA'
                    lshapedgap = 'NA'
                    lshapedtime = t2
                    lshapedsave = 'NA'
            except GurobiError:
                lshapedobj = 'Failed'
                lshapedsave = 'Failed'
                lshapedgap = 'Failed'
                lshapedtime = 'Failed'
            #Tabu
            try:    
                tabu, t3 = Mod4(lshaped)
                tabuobj = tabu.best[-1]
                tabusave = tabu.best[-3]
                tabutime = t3
                
            except GurobiError:
                tabuobj = 'Failed'
                tabusave = 'Failed'
                tabutime = 'Failed'
            
            write.writerow((str(rides),
                            str(bus),
                            str(scenarios),
                            str(twostageobj),
                            str(twostagegap),
                            str(twostagetime),
                            str(lshapedobj),
                            str(lshapedsave),
                            str(lshapedgap),
                            str(lshapedtime),
                            str(tabuobj),
                            str(tabusave),
                            str(tabutime)))

Using license file C:\Users\lavkeshg\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter NodefileStart to 0.5
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2624 rows, 1570 columns and 10138 nonzeros
Model fingerprint: 0x42e3525c
Variable types: 624 continuous, 946 integer (946 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-16, 1e+00]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 1e+05]
Presolve removed 493 rows and 548 columns
Presolve time: 0.04s
Presolved: 2131 rows, 1022 columns, 7788 nonzeros
Variable types: 422 continuous, 600 integer (600 binary)

Root relaxation: objective 1.340000e+01, 492 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 4458 rows, 2595 columns and 17606 nonzeros
Model fingerprint: 0x9cf5560c
Variable types: 989 continuous, 1606 integer (1606 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-16, 1e+00]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 1e+05]
Presolve removed 764 rows and 839 columns
Presolve time: 0.06s
Presolved: 3694 rows, 1756 columns, 14020 nonzeros
Variable types: 684 continuous, 1072 integer (1072 binary)

Root relaxation: objective 1.120000e+01, 695 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.20000    0  146          -   11.20000      -     -    0s
     0     0   19.

     0     0   22.52905    0   41   68.33231   22.52905  67.0%     -    1s
     0     0   22.52905    0   41   68.33231   22.52905  67.0%     -    1s
     0     0   22.80888    0   46   68.33231   22.80888  66.6%     -    1s
H    0     0                      52.4776758   22.80888  56.5%     -    1s
     0     0   22.80888    0   46   52.47768   22.80888  56.5%     -    1s
     0     0   22.85612    0   47   52.47768   22.85612  56.4%     -    1s
     0     0   22.85612    0   47   52.47768   22.85612  56.4%     -    1s
     0     0   22.85612    0   38   52.47768   22.85612  56.4%     -    1s
     0     0   22.85612    0   40   52.47768   22.85612  56.4%     -    1s
     0     0   22.95334    0   44   52.47768   22.95334  56.3%     -    1s
     0     0   22.99153    0   47   52.47768   22.99153  56.2%     -    1s
     0     0   22.99200    0   47   52.47768   22.99200  56.2%     -    1s
     0     0   23.06664    0   47   52.47768   23.06664  56.0%     -    1s
     0     0   23.16394  

  3119   915   45.00445   19  197   58.53171   33.88270  42.1%  98.3   20s
  3129   922   39.03821   11  457   58.53171   33.88270  42.1%  98.0   25s
  3142   930   40.47556    7  624   58.53171   33.88270  42.1%  97.6   30s
  3495   991   33.88270   20  411   58.53171   33.88270  42.1%   109   35s
* 3886   993              50      51.9317117   33.88270  34.8%   108   37s
  3929   999   43.69367   31  150   51.93171   33.88270  34.8%   109   43s
* 4223   921              37      51.8317117   33.88270  34.6%   109   44s
  4259   911   37.04990   31  359   51.83171   33.88270  34.6%   109   45s
  4888   940   45.21974   29  229   51.83171   33.88270  34.6%   113   50s
  5328   901   41.33861   48  244   51.83171   33.88270  34.6%   115   57s
H 5333   780                      46.9820026   33.88270  27.9%   115   57s
H 5336   690                      45.4569298   33.88270  25.5%   115   57s
  5718   800   33.88270   34  454   45.45693   33.88270  25.5%   116   60s
* 6312   994             

     0     0   25.50019    0   65  265.57123   25.50019  90.4%     -    2s
     0     0   25.60964    0   69  265.57123   25.60964  90.4%     -    2s
     0     0   25.61350    0   67  265.57123   25.61350  90.4%     -    2s
     0     0   25.75336    0   66  265.57123   25.75336  90.3%     -    2s
     0     0   25.77449    0   66  265.57123   25.77449  90.3%     -    2s
     0     0   25.77906    0   63  265.57123   25.77906  90.3%     -    2s
     0     0   25.83678    0   56  265.57123   25.83678  90.3%     -    2s
     0     0   25.84970    0   57  265.57123   25.84970  90.3%     -    2s
     0     0   25.84979    0   58  265.57123   25.84979  90.3%     -    2s
     0     0   25.91450    0   58  265.57123   25.91450  90.2%     -    3s
     0     0   25.91450    0   58  265.57123   25.91450  90.2%     -    3s
     0     2   26.61135    0   58  265.57123   26.61135  90.0%     -    3s
*  111    80              28     207.6326172   26.61135  87.2%  42.3    4s
*  265   161             

...

-12.862631571477323 -12.490879999999997
22.123413739354177 21.75166216787685
-12.940698380242724 -12.4721
22.220389869621158 21.751791489378434
-12.844900842958468 -12.534549999999998
22.06226194452335 21.75191110156488
-12.90081412922005 -12.617100000000008
22.03573517920075 21.752021049980705
-12.85396925543002 -12.57534
22.030698653166585 21.752069397736566
-12.931335760840351 -12.579320000000006
22.104134905997487 21.752119145157142
-12.89656753197156 -12.515640000000012
22.133093775157448 21.7521662431859
-12.82196714989725 -12.605909999999996
21.96830129753284 21.752244147635587
-12.878153447240422 -12.593029999999997
22.03741711507412 21.752293667833698
-12.799482572315826 -12.586240000000002
21.96560995212709 21.752367379811268
-12.90388582101341 -12.643499999999992
22.0128072707131 21.752421449699682
-12.958238916503129 -12.65746
22.053251073853573 21.752472157350446
-12.9330716445158 -12.534390000000007
22.151207298927538 21.75252565441175
-13.004837840785198 -12.480570000000

-12.797479367220074 -12.714350000000001
21.84153778119976 21.75840841397969
-12.814724651106342 -12.739010000000015
21.834138983632393 21.758424332526065
-12.786117007152702 -12.731070000000006
21.813489456744552 21.758442449591854
-12.894383931887205 -12.774779999999991
21.87806279979072 21.758458867903506
-12.7983044322257 -12.757570000000001
21.79921474866294 21.758480316437243
-12.769285013731572 -12.713689999999996
21.81409528628321 21.758500272551633
-12.738175431386832 -12.613789999999991
21.882893056790106 21.758507625403265
-12.781925362775564 -12.694060000000002
21.846389764314885 21.758524401539322
-12.819454548603803 -12.77283
21.805172814422228 21.758548265818426
-12.831331108947282 -12.754280000000003
21.835616899184878 21.758565790237597
-12.80813432469415 -12.693970000000007
21.87274468752151 21.758580362827367
-12.891694542610821 -12.60273
22.047571333287138 21.758606790676318
-12.874507327129288 -12.660769999999985
21.972421942255963 21.75868461512666
-12.907780352788

     0     0   25.24012    0  357          -   25.24012      -     -  199s
     0     0   25.24323    0  362          -   25.24323      -     -  200s
     0     0   25.24382    0  361          -   25.24382      -     -  201s
     0     0   25.24387    0  360          -   25.24387      -     -  202s
     0     0   25.54338    0  353          -   25.54338      -     -  214s
     0     0   25.55224    0  352          -   25.55224      -     -  215s
     0     0   25.55266    0  354          -   25.55266      -     -  216s
     0     0   25.99819    0  340          -   25.99819      -     -  227s
     0     0   26.02121    0  349          -   26.02121      -     -  229s
     0     0   26.02349    0  348          -   26.02349      -     -  230s
     0     0   26.02363    0  346          -   26.02363      -     -  231s
     0     0   26.21987    0  451          -   26.21987      -     -  240s
     0     0   26.24559    0  450          -   26.24559      -     -  242s
     0     0   26.24698  

  1094   989   36.08443   70  419          -   26.83421      -   251 1731s
  1095   989  240.59981  111  418          -   26.83421      -   250 1738s
  1096   990  134.91155   84  418          -   26.83421      -   250 1746s
  1097   991   39.03925   39  419          -   26.83421      -   250 1753s
  1098   991  100.13551   50  418          -   26.83421      -   250 1761s
  1099   992   36.92605   76  416          -   26.83421      -   249 1768s
  1100   993   33.63381  119  415          -   26.83421      -   249 1826s
  1101   993   44.12853  116  415          -   26.83421      -   249 1835s
  1102   994   31.08026   79  415          -   26.83426      -   249 1856s
  1103   998   26.83426   11  417          -   26.83426      -   283 1970s
  1105  1001   26.83426   12  396          -   26.83426      -   284 2051s
  1109  1004   26.83426   13  399          -   26.83426      -   285 2063s
  1113  1006   27.96780   13  335          -   26.83426      -   288 2089s
  1117  1009   26.85374  

 21639 18598  990.66783  175   56 2375.19464   28.03595  98.8%   174 9360s
*22007 18592             172    2144.7902246   28.04930  98.7%   174 9443s
 22176 18766   30.35109   27  203 2144.79022   28.05357  98.7%   174 9494s
 22380 18935   32.81336   40  138 2144.79022   28.05357  98.7%   175 9547s
H22381 18933                    2080.4900442   28.05357  98.7%   175 9880s
 22549 19201   34.87540   62  113 2080.49004   28.05357  98.7%   177 9936s
 22852 19284   72.94657   89   97 2080.49004   28.05357  98.7%   178 9995s
H22867 19023                    1180.5342963   28.05357  97.6%   178 10006s
 22952 19129   73.64191   97   90 1180.53430   28.05357  97.6%   179 10112s
H22967 18382                     737.8147536   28.05357  96.2%   179 10135s
 23085 18436   74.49021  105   97  737.81475   28.05357  96.2%   179 10252s
 23190 18558   77.39190  118   98  737.81475   28.05357  96.2%   180 10346s
H23216 17690                     541.9323120   28.05357  94.8%   180 10477s
H23325 17596       

In [ ]:
MIPGap = 0.001
tests = [(3,1),(4,1),(5,1),(6,2),(7,2),(8,2),(9,2)]
scenarioslist = [10,15,20]
for s in scenarioslist:
    scenarios = s
    for test in tests:
        rides, bus = test
        mappy = Map(rides)        
#         drpstw, t1 = Mod1()
#         objdrpstw = drpstw.model.ObjVal
        twosp, t2 = Mod2()
        objtwosp = twosp.model.ObjVal
        lshaped, t3 = Mod3() 
        objlshaped = lshaped.model.ObjVal
#         tabu, t4 = Mod4()
#         objtabu = tabu.best[-2]
        write = [
#             str(objdrpstw), str(t1),
            str(objtwosp), str(t2),
            str(objlshaped), str(t3),
#             str(objtabu), str(t4),
            str(scenarios),str(bus),str(rides)]
        with open('report.txt', 'a') as file:
            file.write('\n')
            for w in write:
                file.write(w)
                file.write(',')

### DARP-STW Model

#### Model Solution 

In [ ]:
drpstw, t1 = Mod1()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(drpstw.bus)}
for i, j in drpstw.parameters.edges:
    for k in range(drpstw.bus):
        if drpstw.variables.x[i,j,k].X > 0.5:
            if j != drpstw.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objdrpstw = drpstw.model.ObjVal
displaygraph(nodes, edges,'DARP-STW')
nodes,edges

### Two-Stage Stochastic Model

#### Model Solution

In [ ]:
twosp, t2 = Mod2()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(twosp.bus)}
for i, j in twosp.parameters.edges:
    for k in range(twosp.bus):
        if twosp.MP.variables.x[i,j,k].X > 0.5:
            if j != twosp.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))

objtwosp = twosp.model.ObjVal
displaygraph(nodes, edges,'Two-Stage')
nodes,edges

### L-Shaped Method

#### Model Solution

In [ ]:
lshaped, t3 = Mod3()

In [ ]:
lshaped.LazyInt

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.variables.x[i,j,k].X > 0.5:
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges,bounds

In [ ]:
plot_trend(lshaped.upperbounds[2:],lshaped.lowerbounds[2:],0)

In [ ]:
mostcancel = {i: 0 for i in lshaped.parameters.pickup}
for i in lshaped.parameters.pickup:
    for s in range(lshaped.scenarios):
        mostcancel[i] += 2 - lshaped.submodel[s].sim.alpha[i] - lshaped.submodel[s].sim.alpha[i+lshaped.parameters.rides]
mostcancel = sorted(mostcancel.items(), key=lambda x: (x[1]))
mostcancel

### Tabu-Heuristic

#### Model Solution

In [ ]:
tabu, t4 = Mod4()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabu.bus:
    for i in tabu.best[k]:
        n = tabu.best[k][i]
        if i != tabu.model.last:
            edges[k].append((i,n.next.key))
objtabu = tabu.best[-2]
displaygraph(nodes, edges,'Tabu')    
nodes,edges, tabu.best

In [ ]:
tabutsp, t5 = Mod5()

In [ ]:
nodes = mappy.node
edges = {k:[] for k in tabu.bus}
for k in tabutsp.bus:
    for i in tabutsp.best[k]:
        n = tabutsp.best[k][i]
        if i != tabutsp.model.last:
            edges[k].append((i,n.next.key))
objtabutsp = tabutsp.best[-2]
displaygraph(nodes, edges,'TabuTSP')    
nodes,edges, tabutsp.best

In [ ]:
print(t1,t2,t3,t4,t5)
objdrpstw,objtwosp,objlshaped,objtabu,objtabutsp

In [ ]:
mappy.pickup_time

In [ ]:
# print(twosp.sim.alpha)
lshaped.printsol(lshaped.submodel)

In [ ]:
nodes = mappy.node
edges = {k:[] for k in range(lshaped.bus)}
for i, j in lshaped.parameters.edges:
    for k in range(lshaped.bus):
        if lshaped.submodel[3].variables.xs[i,j,k].X > 0.5:
            if i == 0 and j == 9:
                continue
            if j != lshaped.last:
                edges[k].append((i,j))
            else:
                edges[k].append((i,0))
bounds = lshaped.getcancel()
objlshaped = lshaped.model.ObjVal
displaygraph(nodes, edges,'L-Shaped')
nodes,edges

In [ ]:
drpstw